In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk.tokenize as tokenize
import re
import string
from collections import defaultdict

In [3]:
df=pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df=df.iloc[:1000,:]
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

TEXT CLEANING

In [4]:
df["sentiment"].replace({"positive":1,"negative":0}, inplace =True)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
import string #for punctuation
import nltk #natural language tool kit
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

print(string.punctuation)
print('------------------------------------------------------------------------------------------------------------------------')
print(stopwords.words('english'))
redundant_words=['br']
def text_cleaning(sentence): #sentence is text(paragraph)
    sentence_punc_removed=[letter for letter in sentence if letter not in string.punctuation]
    sentence_punc_removed=''.join(sentence_punc_removed) # punctuation removed
    #stopwords are filtered out
    sentence_clean=[word for word in sentence_punc_removed.split() if word.lower() not in stopwords.words('english')]
    # redundant words are filtered out
    sentence_clean=[word for word in sentence_clean if word.lower() not in redundant_words ] ##array of words of that sentence
    # lemmatization is done here.
    final_sentence = [lemmatizer.lemmatize(word.lower()) for word in sentence_clean]
    return final_sentence # ARRAY OF WORDS

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
------------------------------------------------------------------------------------------------------------------------
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'wher

In [6]:
df_clean=df['review'].apply(text_cleaning)

TOKENIZATION / COUNT VECTORIZER

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

#here, we performed data cleaning and count vectorization sequentially altogether !

vectorizer=CountVectorizer(analyzer=text_cleaning,dtype='uint8').fit_transform(df['review']) #transforms text into numeric vectorized format

BUILDING MODEL

In [8]:
print(type(vectorizer)) 
X=vectorizer.toarray() 
y=df['sentiment']
print(X.shape)


<class 'scipy.sparse.csr.csr_matrix'>
(1000, 18917)


In [9]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state = 42) 
X_test.shape,X_train.shape,y_test.shape,y_train.shape


((200, 18917), (800, 18917), (200,), (800,))

In [10]:
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
my_model = XGBClassifier()
my_model.fit(X_train, y_train)
predictions = my_model.predict(X_test)
cm=confusion_matrix(y_test,predictions)
cm

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:41:46] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([[79, 25],
       [18, 78]])

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)


0.785